In [1]:
from fileinput import filename
import os
import random

import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from utils import settings as settings
import pandas as pd
from evaluation.metrics import accuracy
from evaluation.conf_matrix import create_conf_matrix
from loader.Preprocessor import Preprocessor
from models.JensModel import JensModel
from models.RainbowModel import RainbowModel
from models.ResNetModel import ResNetModel
from utils.filter_activities import filter_activities
from utils.folder_operations import new_saved_experiment_folder
from utils.DataConfig import Sonar22CategoriesConfig, OpportunityConfig, SonarConfig, LabPoseConfig
from tensorflow.keras.layers import (Dense)
from utils.Recording import Recording
import matplotlib.pyplot as plt

2022-05-25 01:01:43.615767: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 01:01:43.615792: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import utils.DataConfig
import importlib
importlib.reload(utils.DataConfig)
from utils.DataConfig import Sonar22CategoriesConfig, OpportunityConfig, SonarConfig, LabPoseConfig

# Init
# TODO: refactor, find out why confusion matrix sometimes shows different results than accuracy
# TODO: - make train and test datasets evenly distributed
#       - make 
""" 
Number of recordings per person
{'connie.csv': 6, 'alex.csv': 38, 'trapp.csv': 9, 'anja.csv': 13, 'aileen.csv': 52, 'florian.csv': 16, 'brueggemann.csv': 36, 'oli.csv': 20, 'rauche.csv': 9, 'b2.csv': 6, 'yvan.csv': 8, 'christine.csv': 7, 'mathias.csv': 2, 'kathi.csv': 17}
"""

data_config = LabPoseConfig(dataset_path='/dhc/groups/bp2021ba1/data/lab_data')#OpportunityConfig(dataset_path='/dhc/groups/bp2021ba1/data/opportunity-dataset')
data_config = SonarConfig(dataset_path='/dhc/groups/bp2021ba1/data/lab_data_alex')#OpportunityConfig(dataset_path='/dhc/groups/bp2021ba1/data/opportunity-dataset')
settings.init(data_config)
random.seed(1678978086101)

k_fold_splits = 3
numEpochsBeforeTL = 10
numEpochsForTL = 3
minimumRecordingsPerLeftOutPerson = 5
# Load dataset
recordings = settings.DATA_CONFIG.load_dataset()#limit=75


/dhc/groups/bp2021ba1/data/lab_data_alex/1651065958950
took: 502.109 - 0.091 - 4.678 - 12.961
1.0
/dhc/groups/bp2021ba1/data/lab_data_alex/1651139232298
took: 530.206 - 0.087 - 8.765 - 12.169
1.0


In [7]:
import pose_sequence_loader
import importlib
importlib.reload(pose_sequence_loader)

from pose_sequence_loader import *
from multiprocessing import Pool

def process_recording(index_recording):
    i = index_recording[0]
    recording = index_recording[1]
    append_poseframe(recording)
    print(f"Pose Frame added to Recording {i}/{len(recordings)}  ")


pool = Pool()
pool.imap_unordered(
    process_recording, list(enumerate(recordings)), 1
)
pool.close()
pool.join()
   

2355
2355
2942
2942
0
0
Pose Frame added to Recording 1/2  


In [39]:
recording = recordings[1]

activities = recording.activities
first_activity = activities[0]
for i in range(len(activities)):
    if first_activity != activities[i]:
        break

frame_first_time = recording.time_frame[0]
frame_time_second_activity = recording.time_frame[i]
frame_diff = round((frame_time_second_activity - frame_first_time) / 1000)

print(i, frame_diff)

1547 25784


In [6]:
recordings[0].time_frame

0        384379410
1        384396077
2        384412744
3        384429411
4        384446078
           ...    
19876    715652702
19877    715669369
19878    715686036
19879    715702703
19880    715719370
Name: SampleTimeFine, Length: 19881, dtype: int64